In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

--2019-02-14 12:17:48--  http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.16.64
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.16.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10367709 (9.9M) [application/json]
Saving to: ‘./pw-data/201701scripts_sample.json.gz’

     0K .......... .......... .......... .......... ..........  0% 50.6M 0s
    50K .......... .......... .......... .......... ..........  0% 40.3M 0s
   100K .......... .......... .......... .......... ..........  1% 57.2M 0s
   150K .......... .......... .......... .......... ..........  1% 54.6M 0s
   200K .......... .......... .......... .......... ..........  2% 58.0M 0s
   250K .......... .......... .......... .......... ..........  2% 50.5M 0s
   300K .......... .......... .......... .......... ..........  3% 55.0M 0s
   350K

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [4]:
import gzip
import simplejson as json

In [5]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [107]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [7]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains 
- quantitative data on the number of items dispensed (`'items'`), 
- the total quantity of item dispensed (`'quantity'`), 
- the net cost of the ingredients (`'nic'`), 
- and the actual cost to the patient (`'act_cost'`). 

Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [7]:
import math
import numpy as np
import statistics
N = len(scripts)
mean = sum([scripts[x]['items'] for x in range(N)]) / N
print(mean)
print(math.sqrt(sum([(scripts[x]['items'] - mean)**2 for x in range(N)])/(N-1)))
#sortedlist_ = sorted([scripts[x]['items'] for x in range(len(scripts)-1)])
sortedlist_ = [1, 3, 3, 4, 5, 6, 6, 7, 8, 8]
N = len(sortedlist_)
med = (sortedlist_[math.ceil((N+1)/2)-1] + sortedlist_[math.floor((N+1)/2)-1]) / 2
half1 = [sortedlist_[x] for x in range(N) if sortedlist_[x] <= med]
print("half1:",half1)
half2 = [sortedlist_[x] for x in range(N) if sortedlist_[x] >= med]
print("half2:",half2)
n1 = len(half1)
n2 = len(half2)
print("med:",med)
print("q25:",((half1[math.ceil((n1+1)/2)-1] + half1[math.floor((n1+1)/2)-1]) / 2))#+((med - math.floor(med)))
print("q75:",((half2[math.ceil((n2+1)/2)-1] + half2[math.floor((n2+1)/2)-1]) / 2))#-((med - math.floor(med)))
np_list = np.array(sortedlist_)
print("np_med:",np.percentile(np_list,50))
print("np_q25:",np.percentile(np_list,25))
print("np_q75:",np.percentile(np_list,75))

11.522744731217633
33.11220959820685
half1: [1, 3, 3, 4, 5]
half2: [6, 6, 7, 8, 8]
med: 5.5
q25: 3.0
q75: 7.0
np_med: 5.5
np_q25: 3.25
np_q75: 6.75


In [11]:
import math
def describe(key):
    N = len(scripts)
    list_ = [scripts[x][key] for x in range(N)]
    total = sum(list_)
    avg = total / N
    s = math.sqrt(sum([(scripts[x][key] - avg)**2 for x in range(N)])/(N-1))
    sortedlist_ = sorted(list_)
    med = (sortedlist_[math.ceil((N+1)/2)-1] + sortedlist_[math.floor((N+1)/2)-1]) / 2
    #q25 = np.percentile(sortedlist_,25)
    #q75 = np.percentile(sortedlist_,75)
    half1 = [sortedlist_[x] for x in range(N) if sortedlist_[x] <= med]
    half2 = [sortedlist_[x] for x in range(N) if sortedlist_[x] >= med]
    n1 = len(half1)
    n2 = len(half2)
    half2 = [sortedlist_[x] for x in range(N) if sortedlist_[x] > med]
    q25 = ((half1[math.ceil((n1+1)/2)-1] + half1[math.floor((n1+1)/2)-1]) / 2)
    q75 = ((half2[math.ceil((n2+1)/2)-1] + half2[math.floor((n2+1)/2)-1]) / 2)

    return (total, avg, s, q25, med, q75)

In [12]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [13]:
grader.score.pw__summary_statistics(summary)

Your score:  0.9166666666666667


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [5]:
check = []
bnf_names = []
for x in range(len(scripts)):
    if scripts[x]['bnf_name'] not in check:
        check.append(scripts[x]['bnf_name'])
        bnf_names.append(scripts[x]['bnf_name'])

In [11]:
bnf_names

['Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'Alginate_Raft-Forming Oral Susp S/F',
 'Sod Algin/Pot Bicarb_Susp S/F',
 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
 'Gaviscon Infant_Sach 2g (Dual Pack) S/F',
 'Gaviscon Advance_Liq (Aniseed) (Reckitt)',
 'Gaviscon Advance_Tab Chble Mint(Reckitt)',
 'Gaviscon Advance_Liq (Peppermint) S/F',
 'Peptac_Liq (Peppermint) S/F',
 'Alverine Cit_Cap 60mg',
 'Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp',
 'Hyoscine Butylbrom_Tab 10mg',
 'Mebeverine HCl_Tab 135mg',
 'Mebeverine HCl_Cap 200mg M/R',
 'Peppermint Oil_Cap E/C 0.2ml',
 'Peppermint Oil_Cap E/C 0.2ml M/R',
 'Colpermin_Cap E/C 0.2ml M/R',
 'Ispag/Mebeverine_Gran Eff 3.5g/135mg S/F',
 'Fybogel Mebeverine_Eff Gran Sach S/F',
 'Ranitidine HCl_Tab 150mg',
 'Ranitidine HCl_Tab 300mg',
 'Ranitidine HCl_Oral Soln 75mg/5ml S/F',
 'Esomeprazole_Tab E/C 20mg',
 'Esomeprazole_Tab E/C 40mg',
 'Emozul_Cap E/C 20mg',
 'Emozul_Cap E/C 40mg',
 'Lansoprazole_Cap 30mg (E/C Gran)',
 'Lansoprazole_Cap 15mg (E/C Gran)',

In [15]:
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [6]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    for key in groups.keys():
        if script['bnf_name'] == key:
            groups[key].append(script)
            break

In [73]:
groups['3m Health Care_Cavilon Durable Barrier C'][:5]

[{'bnf_code': '23803108006',
  'items': 3,
  'practice': 'N81013',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 24.36,
  'act_cost': 22.58,
  'quantity': 3},
 {'bnf_code': '23803108011',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 7.96,
  'act_cost': 7.37,
  'quantity': 2},
 {'bnf_code': '23803108006',
  'items': 7,
  'practice': 'N81029',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 64.96,
  'act_cost': 60.23,
  'quantity': 8},
 {'bnf_code': '23803108006',
  'items': 4,
  'practice': 'N81062',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 32.48,
  'act_cost': 30.12,
  'quantity': 4},
 {'bnf_code': '23803108011',
  'items': 1,
  'practice': 'N81062',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 3.98,
  'act_cost': 3.68,
  'quantity': 1}]

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [7]:
q = [(key, sum([groups[key][x]['items']
     for x in range(len(groups[key]))])) for key in groups.keys()]
q_ = [q[x][1] for x in range(len(q))]
max_item = [q[q_.index(max(q_))]]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [69]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [8]:
def group_by_field(data, fields):
    byfields = {field: [] for field in fields}
    groups = {field: {} for field in fields}
    for key in byfields.keys():
        check = []
        for x in range(len(data)):
            if data[x][key] not in check:
                check.append(data[x][key])
                byfields[key].append(data[x][key])
    for field in byfields.keys():
        byfieldsgroups = {name: [] for name in byfields[field]}
        for item in data:
            for name in byfieldsgroups.keys():
                if item[field] == name:
                    byfieldsgroups[name].append(item)
                    break
        
        groups[field] = byfieldsgroups
    
    return groups

In [6]:
def group_by_field_(data, fields):
    for field in fields:
        check = []
        names = []
        for x in range(len(data)):
            if data[x][field] not in check:
                check.append(data[x][field])
                names.append(data[x][field])
        groups = {name: [] for name in names}
        for item in data:
            for key in groups.keys():
                if item[field] == key:
                    groups[key].append(item)
                    break
        
        #data = 
    return groups

In [10]:
groups_ = group_by_field_(scripts, ('bnf_name',))

In [114]:
groups_.keys()

5619

In [11]:
groups = group_by_field(scripts, ('bnf_name',))

In [72]:
groups_['bnf_name']['3m Health Care_Cavilon Durable Barrier C'][:5]

[{'bnf_code': '23803108006',
  'items': 3,
  'practice': 'N81013',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 24.36,
  'act_cost': 22.58,
  'quantity': 3},
 {'bnf_code': '23803108011',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 7.96,
  'act_cost': 7.37,
  'quantity': 2},
 {'bnf_code': '23803108006',
  'items': 7,
  'practice': 'N81029',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 64.96,
  'act_cost': 60.23,
  'quantity': 8},
 {'bnf_code': '23803108006',
  'items': 4,
  'practice': 'N81062',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 32.48,
  'act_cost': 30.12,
  'quantity': 4},
 {'bnf_code': '23803108011',
  'items': 1,
  'practice': 'N81062',
  'bnf_name': '3m Health Care_Cavilon Durable Barrier C',
  'nic': 3.98,
  'act_cost': 3.68,
  'quantity': 1}]

In [13]:
def max_items(groups):
    q = [(key, sum([groups[key][x]['items'] for x in range(len(groups[key]))])) for key in groups.keys()]
    q_ = [q[x][1] for x in range(len(q))]
    
    return ([q[q_.index(max(q_))]])

In [14]:
test_max_item = max_items(groups_)

In [15]:
assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each **postal code**, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [9]:
practice_postal = {}
for practice in practices:
    # if practice code already listed
    if practice['code'] in practice_postal:
        # choose alphabetically 1st postcode
        # only if the alphabetically 1st postcode is the one in practices do we then append to dict with chosen postcodes
        if practice['post_code'] == sorted([practice['post_code'], practice_postal[practice['code']]])[0]:  
            practice_postal[practice['code']] = practice['post_code']  
    else: # each time a new practice code appears
        practice_postal[practice['code']] = practice['post_code']

In [17]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [11]:
newgroups = group_by_field(practices, ('code',))

In [19]:
newgroups['code']['K82019'][:5]

[{'code': 'K82019',
  'name': 'THE MANDEVILLE PRACTICE',
  'addr_1': 'THE MANDEVILLE PRACTICE',
  'addr_2': 'HANNON ROAD',
  'borough': 'AYLESBURY',
  'village': 'BUCKINGHAMSHIRE',
  'post_code': 'HP21 8TR'},
 {'code': 'K82019',
  'name': 'THE MANDEVILLE PRACTICE',
  'addr_1': 'FISHERMEAD MEDICAL CENTRE',
  'addr_2': 'FISHERMEAD BOULEVARD',
  'borough': 'MILTON KEYNES',
  'village': 'BUCKINGHAMSHIRE',
  'post_code': 'MK6 2LR'}]

In [12]:
# create list of selected alphabetically 1st postcodes ordered by code
postcodes = [sorted([newgroups['code'][key][x]['post_code'] 
                 for x in range(len(newgroups['code'][key]))])[0] for key in newgroups['code'].keys()]
# initialise empty dict ordered by 'code'
practice_postal_ = {code : [] for code in newgroups['code']}
# populate dict with list according to the 'code' key
ix=0
for code in practice_postal_:
    practice_postal_[code] = postcodes[ix]
    ix+=1

In [22]:
assert practice_postal_['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [13]:
joined = scripts
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [52]:
items_by_post = group_by_field(joined, ('post_code',))

In [53]:
postal_totals = [(postcode, sum([items_by_post['post_code'][postcode][x]['items']
     for x in range(len(items_by_post['post_code'][postcode]))])) 
                 for postcode in sorted(items_by_post['post_code'].keys())[:100]]#only 1st 100 alphabetically ordered pcodes

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [14]:
items_by_post_ = group_by_field_(joined, ('post_code',))

In [29]:
total_items_by_bnf_post = {}
for postcode in items_by_post_.keys():
    groups = group_by_field_(items_by_post_[postcode], ('bnf_name',))
    for bnf_name in groups.keys():
        total_items_by_bnf_post[(postcode, bnf_name)] = sum([groups[bnf_name][x]['items'] 
                                                         for x in range(len(groups[bnf_name]))]) 

In [42]:
sorted(total_items_by_bnf_post.keys())[:5]

[('B11 4BW', '3m Health Care_Cavilon Durable Barrier C'),
 ('B11 4BW', '3m Health Care_Cavilon No Sting Barrier'),
 ('B11 4BW', 'Abidec_Dps'),
 ('B11 4BW', 'Acetic Acid_Ear Spy 2% 5ml'),
 ('B11 4BW', 'Aciclovir_Crm 5%')]

In [43]:
total_items_by_bnf_post[('B11 4BW', '3m Health Care_Cavilon Durable Barrier C')]

7

In [31]:
assert len(total_items_by_bnf_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

**outdated instructions?**

In [44]:
total_items_by_post = {postcode : [] for postcode in items_by_post_.keys()}    
for key, totals in total_items_by_bnf_post.items():
    if key[0] in total_items_by_post:
        total_items_by_post[key[0]].append((key[1],totals))
    else:
        total_items_by_post[postcode].append((key[1],totals))

In [45]:
total_items_by_post.keys()

dict_keys(['SK11 6JL', 'CW5 5NX', 'CW1 3AW', 'CW7 1AT', 'CH65 6TG', 'CH1 4DS', 'TS10 4NW', 'TS24 7PW', 'TS17 0EE', 'TS23 2DG', 'TS3 6AL', 'TS1 2NX', 'OL4 1YN', 'OL1 1NL', 'BL9 0SN', 'WN7 1HR', 'BL3 5HP', 'BL1 8TU', 'M26 2SP', 'BL9 0NJ', 'M35 0AD', 'OL9 7AY', 'OL11 1DN', 'M30 0NU', 'M11 4EJ', 'SK6 1ND', 'WN3 5HL', 'WN2 5NG', 'WN7 2PE', 'BB2 1AX', 'BB3 1PY', 'FY2 0JG', 'FY4 1TJ', 'BB11 2DL', 'BB8 0JZ', 'BB9 7SR', 'BB7 2JG', 'BB4 5SL', 'LA1 1PN', 'FY5 2TZ', 'FY7 8GU', 'WA7 1AB', 'L36 7XY', 'L31 0DJ', 'WA10 2DJ', 'WA9 1LN', 'L7 6HD', 'L5 0QW', 'NE24 1DX', 'NE10 9QG', 'NE38 7NQ', 'SR4 7XF', 'NE37 2PU', 'SR5 2LT', 'YO16 4LZ', 'HG1 5AR', 'HU7 4DW', 'DN34 4GB', 'S63 9EH', 'S74 9AF', 'DN22 7XF', 'S65 1DA', 'WF13 1HN', 'BD3 8QH', 'LS9 9EF', 'CV1 4FS', 'B11 4BW', 'B21 9RY', 'B18 7AL', 'B70 7AW', 'WV13 2DR', 'WS10 8SY', 'WS3 3JP', 'B23 6DJ', 'NG7 3GW', 'NG7 5HY', 'TN24 0GP', 'CB9 8HF', 'SS13 3HQ', 'SS8 0JA', 'SS9 5UU', 'SS0 7AF', 'NN16 8DN', 'WD18 0JP', 'LN2 2JP', 'LE18 2EW', 'LE5 3GH', 'LE10 1DS'

In [46]:
total_items_by_post['SK11 6JL']

[('Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 5),
 ('Alginate_Raft-Forming Oral Susp S/F', 3),
 ('Sod Algin/Pot Bicarb_Susp S/F', 94),
 ('Sod Alginate/Pot Bicarb_Tab Chble 500mg', 9),
 ('Gaviscon Infant_Sach 2g (Dual Pack) S/F', 41),
 ('Gaviscon Advance_Liq (Aniseed) (Reckitt)', 98),
 ('Gaviscon Advance_Tab Chble Mint(Reckitt)', 16),
 ('Gaviscon Advance_Liq (Peppermint) S/F', 65),
 ('Peptac_Liq (Peppermint) S/F', 14),
 ('Alverine Cit_Cap 60mg', 10),
 ('Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp', 3),
 ('Hyoscine Butylbrom_Tab 10mg', 153),
 ('Mebeverine HCl_Tab 135mg', 84),
 ('Mebeverine HCl_Cap 200mg M/R', 194),
 ('Peppermint Oil_Cap E/C 0.2ml', 42),
 ('Peppermint Oil_Cap E/C 0.2ml M/R', 5),
 ('Colpermin_Cap E/C 0.2ml M/R', 6),
 ('Ispag/Mebeverine_Gran Eff 3.5g/135mg S/F', 10),
 ('Fybogel Mebeverine_Eff Gran Sach S/F', 8),
 ('Ranitidine HCl_Tab 150mg', 360),
 ('Ranitidine HCl_Tab 300mg', 77),
 ('Ranitidine HCl_Oral Soln 75mg/5ml S/F', 25),
 ('Esomeprazole_Tab E/C 20mg', 70),
 ('Esomeprazole_T

In [49]:
'SK11 6JL', sorted(total_items_by_post['SK11 6JL'], key=lambda pair: pair[1])[-1:]

('SK11 6JL', [('Omeprazole_Cap E/C 20mg', 3219)])

In [105]:
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [66]:
max_item_by_post = [(postcode, sorted(group, key=lambda pair: pair[1])[-1]) 
                    for postcode, group in total_items_by_post.items()]
sorted(max_item_by_post)

[('B11 4BW', ('Salbutamol_Inha 100mcg (200 D) CFF', 706)),
 ('B18 7AL', ('Salbutamol_Inha 100mcg (200 D) CFF', 556)),
 ('B21 9RY', ('Metformin HCl_Tab 500mg', 1033)),
 ('B23 6DJ', ('Lansoprazole_Cap 30mg (E/C Gran)', 599)),
 ('B70 7AW', ('Paracet_Tab 500mg', 975)),
 ('BB11 2DL', ('Omeprazole_Cap E/C 20mg', 991)),
 ('BB2 1AX', ('Omeprazole_Cap E/C 20mg', 1030)),
 ('BB3 1PY', ('Omeprazole_Cap E/C 20mg', 1869)),
 ('BB4 5SL', ('Omeprazole_Cap E/C 20mg', 1196)),
 ('BB7 2JG', ('Omeprazole_Cap E/C 20mg', 1314)),
 ('BB8 0JZ', ('Atorvastatin_Tab 20mg', 1227)),
 ('BB9 7SR', ('Omeprazole_Cap E/C 20mg', 911)),
 ('BD3 8QH', ('Atorvastatin_Tab 40mg', 719)),
 ('BH18 8EE', ('Omeprazole_Cap E/C 20mg', 1143)),
 ('BH23 3AF', ('Omeprazole_Cap E/C 20mg', 1215)),
 ('BL1 8TU', ('Omeprazole_Cap E/C 20mg', 809)),
 ('BL3 5HP', ('Omeprazole_Cap E/C 20mg', 912)),
 ('BL9 0NJ', ('Omeprazole_Cap E/C 20mg', 1078)),
 ('BL9 0SN', ('Omeprazole_Cap E/C 20mg', 1149)),
 ('CB9 8HF', ('Omeprazole_Cap E/C 20mg', 1841)),
 ('CH

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [65]:
postal_totals = [(postcode, sum([items_by_post_[postcode][x]['items']
     for x in range(len(items_by_post_[postcode]))])) 
                 for postcode in sorted(items_by_post_.keys())]
postal_totals

[('B11 4BW', 20673),
 ('B18 7AL', 19001),
 ('B21 9RY', 29103),
 ('B23 6DJ', 24859),
 ('B70 7AW', 36531),
 ('BB11 2DL', 34356),
 ('BB2 1AX', 28254),
 ('BB3 1PY', 54514),
 ('BB4 5SL', 29388),
 ('BB7 2JG', 44585),
 ('BB8 0JZ', 54380),
 ('BB9 7SR', 38224),
 ('BD3 8QH', 21010),
 ('BH18 8EE', 39413),
 ('BH23 3AF', 32545),
 ('BL1 8TU', 26132),
 ('BL3 5HP', 27147),
 ('BL9 0NJ', 32062),
 ('BL9 0SN', 35275),
 ('CB9 8HF', 51337),
 ('CH1 4DS', 34915),
 ('CH65 6TG', 25090),
 ('CT11 8AD', 44358),
 ('CV1 4FS', 37210),
 ('CW1 3AW', 64104),
 ('CW5 5NX', 38797),
 ('CW7 1AT', 43164),
 ('DA1 2HA', 26075),
 ('DA11 8BZ', 24090),
 ('DN22 7XF', 43091),
 ('DN34 4GB', 48013),
 ('FY2 0JG', 69118),
 ('FY4 1TJ', 62886),
 ('FY5 2TZ', 44258),
 ('FY7 8GU', 34473),
 ('GL1 3PX', 38120),
 ('GL50 4DP', 74822),
 ('GU9 9QS', 32131),
 ('HA0 4UZ', 22755),
 ('HA3 7LT', 32113),
 ('HG1 5AR', 32684),
 ('HU7 4DW', 49107),
 ('KT14 6DH', 26758),
 ('KT6 6EZ', 38975),
 ('L31 0DJ', 32065),
 ('L36 7XY', 22965),
 ('L5 0QW', 24676),
 ('L

In [72]:
postal_totals[0][1]

20673

In [74]:
items_by_region = [(postal_totals[x][0], sorted(max_item_by_post)[x][1][0], (sorted(max_item_by_post)[x][1][1])/postal_totals[x][1]) 
                   for x in range(len(postal_totals))][:100]
#items_by_region

In [75]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*